In [ ]:
import pyodeint
import joblib
import pandas as pd
import random
from chempy import Substance, Reaction, ReactionSystem
from chempy.kinetics.rates import Arrhenius, MassAction
from collections import defaultdict
from chempy.units import SI_base_registry, default_units as u
import numpy as np
from chempy.kinetics.ode import get_odesys
from itertools import product
import csv
import os

In [6]:
database = joblib.load("database_two_reactions.joblib")
database

,Number of Reactions,Step,SM_left,C_left,B_left,P_right,IMP1_right,INT1_right,C_right,B_right,R_left,P_left,INT1_left,IMP2_right,Reaction Format,Initial Compounds
0,2,1,1,0,0,0,0,1,0,0,0,0,0,0,SM -> INT1,SM
1,2,1,1,0,0,0,0,1,0,0,1,0,0,0,SM + R -> INT1,"SM,R"
2,2,1,1,0,0,0,1,0,0,0,0,0,0,0,SM -> IMP1,"SM,IMP1"
3,2,1,1,0,0,0,1,0,0,0,1,0,0,0,SM + R -> IMP1,"SM,R,IMP1"
4,2,1,1,0,0,0,1,1,0,0,0,0,0,0,SM -> IMP1 + INT1,"SM,IMP1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,2,2,1,1,1,0,0,0,0,1,1,1,1,1,SM + P + C + B + R + INT1 -> IMP2 + B,"SM,B,C,R"
586,2,2,1,1,1,0,0,0,1,0,0,1,1,1,SM + P + C + B + INT1 -> IMP2 + C,"SM,B,C"
587,2,2,1,1,1,0,0,0,1,0,1,1,1,1,SM + P + C + B + R + INT1 -> IMP2 + C,"SM,B,C,R"
588,2,2,1,1,1,0,0,0,1,1,0,1,1,1,SM + P + C + B + INT1 -> IMP2 + C + B,"SM,B,C"


In [7]:
valid_combinations_two_reactions = joblib.load("valid_combinations_two_reactions.joblib")
print(valid_combinations_two_reactions)
print(len(valid_combinations_two_reactions))

[(0, 146), (0, 147), (0, 154), (0, 155), (1, 146), (1, 147), (1, 154), (1, 155), (4, 146), (4, 147), (4, 154), (4, 155), (5, 146), (5, 147), (5, 154), (5, 155), (7, 126), (7, 162), (7, 163), (7, 334), (7, 382), (8, 138), (8, 170), (9, 138), (9, 139), (9, 170), (9, 171), (11, 126), (11, 162), (11, 163), (11, 334), (11, 382), (12, 138), (12, 170), (13, 138), (13, 139), (13, 170), (13, 171), (16, 146), (16, 147), (16, 154), (16, 155), (16, 200), (16, 201), (16, 208), (16, 209), (17, 146), (17, 147), (17, 154), (17, 155), (17, 200), (17, 201), (17, 208), (17, 209), (24, 146), (24, 147), (24, 154), (24, 155), (24, 200), (24, 201), (24, 208), (24, 209), (25, 146), (25, 147), (25, 154), (25, 155), (25, 200), (25, 201), (25, 208), (25, 209), (29, 126), (29, 162), (29, 163), (29, 179), (29, 216), (29, 217), (29, 334), (29, 382), (29, 400), (29, 448), (32, 138), (32, 170), (32, 192), (32, 224), (33, 138), (33, 139), (33, 170), (33, 171), (33, 192), (33, 193), (33, 224), (33, 225), (37, 126), (37

In [8]:
equiv_B = [i + 1 for i in range(3)]
print(equiv_B)

[1, 2, 3]


In [9]:
# Generate initial conditions
seed_value = 37
random.seed(seed_value)

T_list = [273, 313, 373]

conc_SM = [0.3] 
conc_IMP2 = 0
conc_IMP3 = 0
conc_IMP4 = 0

equiv_R = [1.2, 1.5]
equiv_B = [3] 
equiv_C = [0.3] 
equiv_P = [0, 0.5]
equiv_IMP1 = [0, 0.002]

imp2_array = np.array(conc_IMP2)
imp3_array = np.array(conc_IMP3)
imp4_array = np.array(conc_IMP4)

c0_list = []
for conc_sm in conc_SM:
    for equiv_r in equiv_R:
        for equiv_b in equiv_B:
            for equiv_c in equiv_C:
                for equiv_p in equiv_P:
                    for equiv_imp1 in equiv_IMP1:
                        sm_array = np.array(conc_sm)
                        r_array = np.array(equiv_r*conc_sm)
                        b_array = np.array(equiv_b*conc_sm)
                        c_array = np.array(equiv_c*conc_sm)
                        p_array = np.array(equiv_p*conc_sm)
                        imp1_array = np.array(equiv_imp1*conc_sm)
                        c0_list.append("c0 = defaultdict(lambda: 0*u.M, {{'SM': {}*u.M, 'R': {}*u.M, 'B': {}*u.M, 'C': {}*u.M, 'P': {}*u.M, 'IMP1': {}*u.M, 'IMP2': {}*u.M}})"\
                                       .format(sm_array, r_array, b_array, c_array, p_array, imp1_array, imp2_array))
print(len(c0_list))

8


In [10]:
c0_list

["c0 = defaultdict(lambda: 0*u.M, {'SM': 0.3*u.M, 'R': 0.36*u.M, 'B': 0.8999999999999999*u.M, 'C': 0.09*u.M, 'P': 0.0*u.M, 'IMP1': 0.0*u.M, 'IMP2': 0*u.M})",
 "c0 = defaultdict(lambda: 0*u.M, {'SM': 0.3*u.M, 'R': 0.36*u.M, 'B': 0.8999999999999999*u.M, 'C': 0.09*u.M, 'P': 0.0*u.M, 'IMP1': 0.0006*u.M, 'IMP2': 0*u.M})",
 "c0 = defaultdict(lambda: 0*u.M, {'SM': 0.3*u.M, 'R': 0.36*u.M, 'B': 0.8999999999999999*u.M, 'C': 0.09*u.M, 'P': 0.15*u.M, 'IMP1': 0.0*u.M, 'IMP2': 0*u.M})",
 "c0 = defaultdict(lambda: 0*u.M, {'SM': 0.3*u.M, 'R': 0.36*u.M, 'B': 0.8999999999999999*u.M, 'C': 0.09*u.M, 'P': 0.15*u.M, 'IMP1': 0.0006*u.M, 'IMP2': 0*u.M})",
 "c0 = defaultdict(lambda: 0*u.M, {'SM': 0.3*u.M, 'R': 0.44999999999999996*u.M, 'B': 0.8999999999999999*u.M, 'C': 0.09*u.M, 'P': 0.0*u.M, 'IMP1': 0.0*u.M, 'IMP2': 0*u.M})",
 "c0 = defaultdict(lambda: 0*u.M, {'SM': 0.3*u.M, 'R': 0.44999999999999996*u.M, 'B': 0.8999999999999999*u.M, 'C': 0.09*u.M, 'P': 0.0*u.M, 'IMP1': 0.0006*u.M, 'IMP2': 0*u.M})",
 "c0 = defa

### Functions Used for Generating Time-Course Data

In [11]:
# Define a function to convert a flat combination into dictionaries
def flat_to_dicts(flat_combination, reagents, products):
    reagent_values = flat_combination[:len(reagents)]
    product_values = flat_combination[len(reagents):]
    
    reagents_dict = {reagent: value for reagent, value in zip(reagents, reagent_values)}
    products_dict = {product: value for product, value in zip(products, product_values)}
    
    return reagents_dict, products_dict

In [12]:
# Convert list to daframe: Initialize dataframe and add data to it
def generate_df_time_course_data(data):
   con_SM, con_R, con_B, con_C, con_P, con_IMP1, con_IMP2, con_INT1 = \
      [], [], [], [], [], [], [], []
   timestamp = [0,0.1,1,20,40,60,120,180,240,300,360,480,600,720,840,960,1080,1200,\
                1500,1800,2400,3000,3600,4500,5400,6300,7200,9000,10800,14400]
   for i in range(0, len(timestamp)):
      con_SM.append("cSM_" + str(timestamp[i]) + "s") 
      con_R.append("cR_" + str(timestamp[i]) + "s") 
      con_B.append("cB_" + str(timestamp[i]) + "s")
      con_C.append("cC_" + str(timestamp[i]) + "s")
      con_P.append("cP_" + str(timestamp[i]) + "s") 
      con_IMP1.append("cIMP1_" + str(timestamp[i]) + "s") 
      con_IMP2.append("cIMP2_" + str(timestamp[i]) + "s")
      con_INT1.append("cINT1_" + str(timestamp[i]) + "s") 
      
   columns = ['Temperature', 'A1', 'Ea1', 'A2', 'Ea2', 'A3', 'Ea3', 'A4', 'Ea4'] + \
      con_SM + con_R + con_B + con_C + con_P + con_IMP1 + con_IMP2 +con_INT1 +  \
         ['Fast_rxn1', 'Medium_rxn1', 'Slow_rxn1', 'Fast_rxn2', 'Medium_rxn2', 'Slow_rxn2',  'Reaction_order', 'Mechanism']
   df_time_course_data = pd.DataFrame(data, columns = columns)
   return df_time_course_data

In [13]:
def generate_order_combinations(reagents, products):
    values_reagents = [1, 2]

    # Generate all possible combinations of values for reagents. The reaction order of product is always 1.
    combinations_reagents = product(values_reagents, repeat=len(reagents))
    product_combination = tuple([1] * len(products))
    
    # Initialize an empty list to store the combinations
    combinations = []
    # Generate combinations for reagents
    for reagent_combination in combinations_reagents:
        # Since products should always be 1, create a tuple of 1s for products
        product_combination = tuple([1] * len(products))
        
        # Combine reagent and product combinations
        combination = list(reagent_combination) + list(product_combination)
        combinations.append(combination)
    return(combinations)

In [14]:
def initialize_csv(filename):
    with open('csv_header.csv', 'r') as header_file:
        csvreader = csv.reader(header_file)
        header = next(csvreader) 
    with open(filename, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(header)  

def write_row_to_csv(filename, row):
    with open(filename, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(row)

### Generate Time Course Data for Two-reaction Mechnism

In [24]:
def generate_time_course_data_two_rxn_csv_out_sim(two_reaction, c0, T, csv_filename, max_file_size=50*1024*1024):

    if not os.path.isfile(csv_filename):
        initialize_csv(csv_filename)
    
    local_vars = {}
    exec(c0, globals(), local_vars)
    c0 = local_vars['c0']

    counter = 0

    for mechanism in two_reaction:
        original_list = database.iloc[mechanism[0],-2].split(' -> ')
        reagents_rxn1 = original_list[0].split(' + ')
        products_rxn1 = original_list[1].split(' + ')
        original_list = database.iloc[mechanism[1],-2].split(' -> ')
        reagents_rxn2 = original_list[0].split(' + ')
        products_rxn2 = original_list[1].split(' + ')
        # print(reagents_rxn1)
        # print(products_rxn1)
        # print(reagents_rxn2)
        # print(products_rxn2)
        
        # Generate all possible combinations for the reaction order
        # Create a list of all possible values for reaction order (1 and 2)
        values = [1, 2]

        # Generate all possible combinations of values for reagents. The reaction order of product is always 1.
        combinations_rxn1 = generate_order_combinations(reagents_rxn1, products_rxn1)
        combinations_rxn2 = generate_order_combinations(reagents_rxn2, products_rxn2)

    
        # Find the order of species in time course data
        all_species = reagents_rxn1 + products_rxn1 + reagents_rxn2 + products_rxn2
        # Merge all lists and remove duplicates
        merged_list = list(set(all_species))
        # Sort the merged list alphabetically
        merged_list.sort()
        # Create a dictionary with items as keys and their corresponding indexes as values
        species_order = {item: index for index, item in enumerate(merged_list)}

        A1, A2, A3, A4 = [0,0,0,0]
        Ea1, Ea2, Ea3, Ea4 = [0,0,0,0]
        timestamp = [0,0.1,1,20,40,60,120,180,240,300,360,480,600,720,840,960,1080,1200,1500,1800,2400,3000,3600,4500,5400,6300,7200,9000,10800,14400]
        num_rxn = 3
        
        for i, combination1 in enumerate(combinations_rxn1):
            for j, combination2 in enumerate(combinations_rxn2):
                # Generate randome Ea and A1
                # 10 time course data for each category (low, medium, fast)
                for a in range(0, num_rxn):
                    for b in range(0, num_rxn):
                        Ea1 = random.randint(0,200)
                        Ea2 = random.randint(0,200)
                        rate = [0] * 6
                        # For slow reactions
                        if a < num_rxn/3:
                            A1 = random.uniform(10e-5, 0.1)
                            rate[2] = 1
                        if a >= num_rxn/3 and a < 2*num_rxn/3:
                            A1 = random.uniform(0.1, 10)
                            rate[1] = 1
                        if a >= 2*num_rxn/3:
                            A1 = random.uniform(10, 1000)
                            rate[0] = 1
                        if b < num_rxn/3:
                            A2 = random.uniform(10e-5, 0.1)
                            rate[5] = 1
                        if b >= num_rxn/3 and b < 2*num_rxn/3:
                            A2 = random.uniform(0.1, 10)
                            rate[4] = 1
                        if b >= 2*num_rxn/3:
                            A2 = random.uniform(10, 1000)
                            rate[3] = 1

                        reagents_dict1, products_dict1 = flat_to_dicts(combination1, reagents_rxn1, products_rxn1)
                        reaction1 = f"r{b}_1 = Reaction({reagents_dict1}, {products_dict1}, MassAction(Arrhenius(unique_keys=('A1', 'Ea_R_1'))))"
                        # print(reaction1)
                        exec(reaction1)
                        
                        reagents_dict2, products_dict2 = flat_to_dicts(combination2, reagents_rxn2, products_rxn2)
                        reaction2 = f"r{b}_2 = Reaction({reagents_dict2}, {products_dict2}, MassAction(Arrhenius(unique_keys=('A2', 'Ea_R_2'))))"
                        # print(reaction2)
                        exec(reaction2)

                        mech = [eval("r%s_1" % b), eval("r%s_2" % b)]
                        rsys = ReactionSystem(mech)

                        # Generate time course data
                        variables = c0.copy()
                        order_rxn1 = eval("r%s_1.order()" % b)
                        order_rxn2 = eval("r%s_2.order()" % b)
                        unit_orderofmag_rxn1 = order_rxn1 - 1
                        unit_orderofmag_rxn2 = order_rxn2 - 1
                        params = {'A1': A1/u.second*u.M**-unit_orderofmag_rxn1, 'Ea_R_1': Ea1/8.3145*u.K**-1, \
                                'A2': A2/u.second*u.M**-unit_orderofmag_rxn2, 'Ea_R_2': Ea2/8.3145*u.K**-1, 'temperature': T*u.K}

                        variables.update(params)
                        rsys.rates(variables)
                        odesys, extra = get_odesys(rsys, include_params=False, lower_bounds=0)
                        # results = odesys.integrate(4*3600*u.s, c0, params, integrator='odeint')
                        try:
                            results = odesys.integrate(4 * 3600 * u.s, c0, params, integrator='odeint', atol=1e-6, rtol=1e-6, nsteps = 5000)
                            conc = results.at(timestamp)
                            res = [T, A1, Ea1, A2, Ea2, A3, Ea3, A4, Ea4]
                            
                            # Append concnetrations
                            all_species = ['SM', 'R', 'B', 'C', 'P', 'IMP1', 'IMP2', 'INT1']
                            for species in all_species:
                                if species in species_order:
                                    for m in range(0, len(timestamp)):
                                        # Remove negative values
                                        if conc[m][0][species_order[species]] > 0:
                                            res.append(conc[m][0][species_order[species]])
                                        else:
                                            res.append(0)
                                else:
                                    res += ([0]* len(timestamp))

                            rxn_order = {'rxn1': reagents_dict1, 'rxn_2': reagents_dict2}
                            res += rate + [rxn_order] + ([mechanism])

                            # write the csv
                            counter += 1
                            write_row_to_csv(csv_filename, res)
                            file_size = os.path.getsize(csv_filename)
                            # print(file_size)
                            # print(max_file_size)
                            if file_size >= max_file_size:    
                                print('yes')
                                import datetime
                                current_datetime = datetime.datetime.now()
                                formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H%M%S")
                                new_filename = f"two_reactions_{formatted_datetime}.csv"
                                # counter +=1
                                initialize_csv(new_filename)
                                csv_filename = new_filename
                        except RuntimeError:
                            print('pass')
                            pass
                        
    print('Number of time course data generated: ', counter)
    # return(data_two_rxn)

    

In [16]:
generate_time_course_data_two_rxn_csv_out_sim(valid_combinations_two_reactions, c0_list[0], T_list[0], 'two_reactions_273K_1.csv')

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
Number of time course data generated:  240408


In [17]:
generate_time_course_data_two_rxn_csv_out_sim(valid_combinations_two_reactions, c0_list[1], T_list[0], 'two_reactions_273K_2.csv')

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
Number of time course data generated:  240408


In [18]:
generate_time_course_data_two_rxn_csv_out_sim(valid_combinations_two_reactions, c0_list[2], T_list[0], 'two_reactions_273K_3.csv')

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
Number of time course data generated:  240408


In [19]:
generate_time_course_data_two_rxn_csv_out_sim(valid_combinations_two_reactions, c0_list[3], T_list[0], 'two_reactions_273K_4.csv')

yes
yes
yes
pass
pass
yes
pass
pass
pass
pass
yes
yes
yes
pass
pass
pass
yes
yes
yes
yes
pass
pass
pass
yes
yes
yes
yes
yes
pass
pass
pass
yes
yes
yes
Number of time course data generated:  240393


In [21]:
generate_time_course_data_two_rxn_csv_out_sim(valid_combinations_two_reactions, c0_list[4], T_list[0], 'two_reactions_273K_5.csv')

yes
yes
yes
pass
pass
yes
pass
pass
pass
pass
pass
yes
yes
pass
pass
pass
yes
yes
yes
yes
pass
pass
pass
pass
pass
yes
pass
yes
yes
yes
yes
pass
yes
yes
yes
Number of time course data generated:  240391


In [22]:
generate_time_course_data_two_rxn_csv_out_sim(valid_combinations_two_reactions, c0_list[5], T_list[0], 'two_reactions_273K_6.csv')

yes
yes
pass
pass
pass
yes
pass
pass
pass
yes
pass
pass
pass
yes
yes
pass
pass
pass
pass
pass
yes
yes
yes
yes
pass
yes
yes
yes
yes
yes
pass
yes
yes
yes
Number of time course data generated:  240392


In [25]:
generate_time_course_data_two_rxn_csv_out_sim(valid_combinations_two_reactions, c0_list[6], T_list[0], 'two_reactions_273K_7.csv')

pass
pass
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
Number of time course data generated:  240406


In [26]:
generate_time_course_data_two_rxn_csv_out_sim(valid_combinations_two_reactions, c0_list[7], T_list[0], 'two_reactions_273K_8.csv')

yes
pass
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
Number of time course data generated:  240407
